In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
from PIL import Image
import os
import random
from sklearn.model_selection import train_test_split
import shutil

import time
import copy
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
from PIL import Image
import os
import random
from sklearn.model_selection import train_test_split
import shutil

import time
import copy

In [3]:
def split_images_sklearn(source_dir, train_dir, val_dir, test_dir, val_size = 0.2, test_size = 0.2, seed=32):
    classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

    for class_name in classes:
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        class_folder = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]

        train_val_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=seed)

        val_ratio = val_size / (1-test_size)
        train_imgs, val_imgs = train_test_split(train_val_imgs, test_size = val_ratio, random_state=seed)

        for img in train_imgs:
            shutil.copy(os.path.join(class_folder, img), os.path.join(train_class_dir, img))
        for img in val_imgs:
            shutil.copy(os.path.join(class_folder, img), os.path.join(val_class_dir, img))
        for img in test_imgs:
            shutil.copy(os.path.join(class_folder, img), os.path.join(test_class_dir, img))

source_dir = r'C:\Users\jagat\Downloads\archive\Garbage classification\Garbage classification'
train_dir = r'Garbage classification/train'
val_dir = r'Garbage classification/val'
test_dir = r'Garbage classification/test'
split_images_sklearn(source_dir, train_dir, val_dir, test_dir)

In [ ]:
# train_compute_transform = transforms.Compose([transforms.Resize((256, 192)),
#                                transforms.ToTensor(),])

# train_dataset = datasets.ImageFolder('Garbage classification/train', transform = train_compute_transform)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4)

: 

: 

In [ ]:
# import torch
# mean = 0.0
# std = 0.0
# nb_samples = 0

# for data, _ in train_loader:
#     batch_samples = data.size(0)
#     data = data.view(batch_samples, data.size(1), -1)

#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     nb_samples += batch_samples

# mean /= nb_samples
# std /= nb_samples

# print(f"Computed Mean: {mean}")
# print(f"Computed Std: {std}")

: 

: 

In [9]:
train_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
train_dataset = datasets.ImageFolder('Garbage classification/train', transform=train_transform)
val_dataset = datasets.ImageFolder('Garbage classification/val', transform=val_transform)
test_dataset = datasets.ImageFolder('Garbage classification/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle=True, num_workers=4)

In [6]:
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}
class_names = train_dataset.classes
num_classes = len(class_names)
print("Number of classes: ", num_classes)
print("Class names: ", class_names)

Number of classes:  6
Class names:  ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [7]:
def train_model(model, criterion, optimizer, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch{epoch+1}/{num_epochs}")

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0
            running_corrects = 0

            for inputs, labels in dataloader:
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best Validation Accuracy: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model

def test_model(model):
    model.eval()
    running_corrects = 0
    for inputs, labels in test_loader:
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / dataset_sizes['test']
    print(f"Test Accuracy: {test_acc:.4f}")
    return test_acc

In [10]:
# models

resnet_model = models.resnet50(pretrained=True)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, num_classes)

c:\Users\jagat\anaconda3\envs\hands_on_ml\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jagat\anaconda3\envs\hands_on_ml\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\jagat/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%



In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
resnet_optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.000055)
num_epochs = 10

In [13]:
print("Training Resnet 50")
resnet_model = train_model(resnet_model, criterion, resnet_optimizer, num_epochs = 5)
print("Testing Resnet 50")
test_model(resnet_model)

Training Resnet 50
Epoch1/5
Train Loss: 0.9242 Acc: 0.6885
Train Loss: 0.9242 Acc: 0.6885
Val Loss: 0.4137 Acc: 0.8659
Epoch2/5
Val Loss: 0.4137 Acc: 0.8659
Epoch2/5
Train Loss: 0.2472 Acc: 0.9339
Train Loss: 0.2472 Acc: 0.9339
Val Loss: 0.3301 Acc: 0.8955
Epoch3/5
Val Loss: 0.3301 Acc: 0.8955
Epoch3/5
Train Loss: 0.1128 Acc: 0.9749
Train Loss: 0.1128 Acc: 0.9749
Val Loss: 0.2373 Acc: 0.9270
Epoch4/5
Val Loss: 0.2373 Acc: 0.9270
Epoch4/5
Train Loss: 0.0722 Acc: 0.9828
Train Loss: 0.0722 Acc: 0.9828
Val Loss: 0.2227 Acc: 0.9329
Epoch5/5
Val Loss: 0.2227 Acc: 0.9329
Epoch5/5
Train Loss: 0.0422 Acc: 0.9927
Train Loss: 0.0422 Acc: 0.9927
Val Loss: 0.2052 Acc: 0.9349
Training complete in 10m 42s
Best Validation Accuracy: 0.9349
Testing Resnet 50
Val Loss: 0.2052 Acc: 0.9349
Training complete in 10m 42s
Best Validation Accuracy: 0.9349
Testing Resnet 50
Test Accuracy: 0.9370
Test Accuracy: 0.9370


tensor(0.9370, dtype=torch.float64)

In [14]:
print("Testing Resnet 50")
test_model(resnet_model)

Testing Resnet 50
Test Accuracy: 0.9370
Test Accuracy: 0.9370


tensor(0.9370, dtype=torch.float64)

In [15]:
print("Train classes:", train_dataset.class_to_idx)
print("Test classes:", test_dataset.class_to_idx)

Train classes: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
Test classes: {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [16]:
import joblib

In [17]:
joblib.dump(resnet_model, 'model.joblib')

['model.joblib']

In [ ]:
# from sklearn.pipeline import Pipeline

# pipe = Pipeline([('restnet_model', resnet_model)])
# pipe.fit(X_train, y_train)

NameError: name 'X_train' is not defined

: 

: 

In [ ]:
# at the end of your training script
torch.save(resnet_model.state_dict(), "model.pt")

: 

: 